In [6]:
import fitz
import pandas as pd
from datetime import datetime
import logging


logging.basicConfig(level=logging.DEBUG)
# filename = 'data/fwil_dhi_me_results_semi.pdf'
# filename = 'data/fwil_dhi_me_results_qr.pdf'
filename = 'data/fwil_dhi_me_results_f.pdf'

In [10]:
doc = fitz.open(filename)
page = doc[0]
text = page.get_text("text")

# Identify where the table starts 25 for the semis and 24 for the qualifiers
text.split('\n')[24:44]


['1. P',
 '1 BRUNI Loic',
 'BRUNI Loic',
 'BRUNI Loic',
 'BRUNI Loic',
 'SPECIALIZED GRAVITY',
 '10007544358',
 'FRA',
 '1994',
 '55.634 (3)',
 '0:46.594 (1)',
 '2:35.483 (1)',
 '3:10.129 (1)',
 '3:40.191 (1)',
 '4:04.264',
 '4:04.264',
 '4:04.264',
 '4:04.264',
 '+0.000',
 '250']

In [9]:
import fitz
from typing import List, Dict, Union
import pandas as pd
from datetime import datetime, timedelta

filename = filename
table_start_line = 25

def extract_time_and_rank(data_string: str) -> (str, str):
    if "(" in data_string:
        time, rank = data_string.split()[0], data_string.split()[-1].strip("()")
        return time, rank
    else:
        return "N/A", "N/A"

def calculate_sector_times(split_times: List[str]) -> List[str]:
    sector_times = []
    previous_time = "0:00.000"

    for split_time in split_times:
        try:
            delta = datetime.strptime(split_time, "%M:%S.%f") - datetime.strptime(previous_time, "%M:%S.%f")
            sector_times.append(str(delta)[2:])  # Skip "0:" part in "0:XX.XXX" string
            previous_time = split_time
        except ValueError:
            sector_times.append("N/A")

    return sector_times

def extract_rider_info_all_pages(filename: str, table_start_line: int = 25) -> List[Dict[str, Union[str, List[str]]]]:
    doc = fitz.open(filename)
    riders_info = []
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")
        lines = text.split('\n')
        line_start = table_start_line

        while line_start < len(lines):
            rider_info = lines[line_start:line_start + 20]
            if len(rider_info) < 19:
                break

            # No team case
            if rider_info[5].isdigit():
                speed_trap, speed_trap_rank = extract_time_and_rank(rider_info[8])
                split_times, split_time_ranks = zip(*(extract_time_and_rank(s) for s in rider_info[9:13]))
                rider_data = {
                    'rank': rider_info[0].split()[0].replace('.', ''),
                    'protected': rider_info[0].split()[1] if len(rider_info[0].split()) > 1 else '',
                    'rider_number': rider_info[1].split()[0],
                    'name': ' '.join(rider_info[1].split()[1:]),
                    'team': 'N/A',
                    'uci_id': rider_info[5],
                    'country': rider_info[6],
                    'birth_year': rider_info[7],
                    'speed_trap': speed_trap,
                    'speed_trap_rank': speed_trap_rank,
                    'split_times': list(split_times),
                    'split_time_ranks': list(split_time_ranks),
                    'final_time': rider_info[13],
                    'gap': rider_info[17] if len(rider_info) > 17 else 'N/A',
                    'points': rider_info[18] if len(rider_info) > 18 else 'N/A'
                }
                next_offset = 19
            # With team case
            else:
                speed_trap, speed_trap_rank = extract_time_and_rank(rider_info[9])
                split_times, split_time_ranks = zip(*(extract_time_and_rank(s) for s in rider_info[10:14]))
                rider_data = {
                    'rank': rider_info[0].split()[0].replace('.', ''),
                    'protected': rider_info[0].split()[1] if len(rider_info[0].split()) > 1 else '',
                    'rider_number': rider_info[1].split()[0],
                    'name': ' '.join(rider_info[1].split()[1:]),
                    'team': rider_info[5],
                    'uci_id': rider_info[6],
                    'country': rider_info[7],
                    'birth_year': rider_info[8],
                    'speed_trap': speed_trap,
                    'speed_trap_rank': speed_trap_rank,
                    'split_times': list(split_times),
                    'split_time_ranks': list(split_time_ranks),
                    'final_time': rider_info[14],
                    'gap': rider_info[18] if len(rider_info) > 18 else 'N/A',
                    'points': rider_info[19] if len(rider_info) > 19 else 'N/A'
                }
                next_offset = 20

            if rider_data['final_time'] in ['DNF', 'DNS']:
                break

            sector_times = calculate_sector_times(rider_data['split_times'])
            rider_data['sector_times'] = sector_times
            riders_info.append(rider_data)
            line_start += next_offset

    return riders_info

# Generate DataFrame
riders_info = extract_rider_info_all_pages(filename, table_start_line)

df = pd.DataFrame(riders_info)
for i in range(4):
    df[f'split_{i+1}'] = df['split_times'].apply(lambda x: x[i] if len(x) > i else 'N/A')
    df[f'split_{i+1}_rank'] = df['split_time_ranks'].apply(lambda x: x[i] if len(x) > i else 'N/A')
    df[f'sector_{i+1}'] = df['sector_times'].apply(lambda x: x[i] if len(x) > i else 'N/A')

# Rank the sector times correctly
for i in range(4):
    df[f'sector_{i+1}_rank'] = df[f'sector_{i+1}'].apply(
        lambda x: timedelta(minutes=int(x.split(":")[0]), seconds=float(x.split(":")[1]))
        if x not in ["N/A", "-"]
        else timedelta.max
    ).rank(method="min").astype(int)

# Handle the final sector (sector_5)
df[f'sector_5'] = df.apply(lambda row: 
    str(datetime.strptime(row["final_time"], "%M:%S.%f") - datetime.strptime(row["split_4"], "%M:%S.%f"))[2:]
    if row["final_time"] not in ["DNF", "DNS", "N/A", "-"] and row["split_4"] not in ["DNF", "DNS", "N/A", "-"]
    else "N/A", axis=1)

df[f'sector_5_rank'] = df[f'sector_5'].apply(
    lambda x: timedelta(minutes=int(x.split(":")[0]), seconds=float(x.split(":")[1]))
    if x not in ["N/A", "-"]
    else timedelta.max
).rank(method="min").astype(int)

df.drop(columns=['split_times', 'split_time_ranks', 'sector_times'], inplace=True)

# Display and Save
display(df)
file_prefix = filename.split('/')[-1].split('.')[0]
df.to_csv(f'data/{file_prefix}.csv', index=False)


,rank,protected,rider_number,name,team,uci_id,country,birth_year,speed_trap,speed_trap_rank,...,sector_3,split_4,split_4_rank,sector_4,sector_1_rank,sector_2_rank,sector_3_rank,sector_4_rank,sector_5,sector_5_rank
0,1,BRUNI,BRUNI,Loic,N/A,10007544358,FRA,1994,55.634,3,...,00:34.646000,3:40.191,1,00:30.062000,1,1,6,3,00:24.073000,10
1,2,P,8,BROSNAN Troy,CANYON CLLCTV FACTORY TEAM,10007307417,AUS,1993,56.031,2,...,00:34.370000,3:41.990,2,00:29.895000,3,3,2,2,00:24.114000,13
2,3,P,4,ILES Finn,SPECIALIZED GRAVITY,10090907774,CAN,1999,55.278,6,...,00:34.963000,3:42.414,3,00:30.130000,4,2,10,4,00:23.839000,2
3,4,P,9,NORTON Dakotah,MONDRAKER FACTORY RACING,10010038167,USA,1992,54.718,9,...,00:34.398000,3:43.253,4,00:30.586000,7,4,3,9,00:24.100000,11
4,5,P,10,SHAW Luca,CANYON CLLCTV FACTORY TEAM,10008813442,USA,1996,54.857,8,...,00:34.754000,3:44.059,5,00:30.349000,2,8,9,8,00:23.884000,4
5,6,P,7,COULANGES Benoit,DORVAL AM COMMENCAL,10008194359,FRA,1994,55.420,5,...,00:34.429000,3:44.528,6,00:29.766000,5,13,4,1,00:23.607000,1
6,7,,38,WILLIAMSON Greg,MADISON SARACEN FACTORY TEAM,10006909111,GBR,1992,56.397,1,...,00:35.660000,3:45.126,7,00:30.148000,11,5,17,5,00:23.949000,7
7,8,P,3,VERGIER Loris,TREK FACTORY RACING GRAVITY,10008723112,FRA,1996,55.208,7,...,00:35.011000,3:45.480,9,00:30.812000,13,6,11,12,00:23.881000,3
8,9,P,5,KOLB Andreas,CONTINENTAL ATHERTON,10009187092,AUT,1996,53.798,13,...,00:35.258000,3:45.373,8,00:30.170000,12,7,14,6,00:24.060000,9
9,10,,18,WALKER Matt,MADISON SARACEN FACTORY TEAM,10011016756,GBR,1999,53.532,18,...,00:34.577000,3:46.006,11,00:31.060000,8,11,5,18,00:24.111000,12


In [5]:
import fitz
from typing import List, Dict, Union
import pandas as pd
from datetime import datetime, timedelta

filename = 'data/fwil_dhi_me_results_qr.pdf'
table_start_line = 24

def extract_time_and_rank(data_string: str) -> (str, str):
    if "(" in data_string:
        time, rank = data_string.split()[0], data_string.split()[-1].strip("()")
        return time, rank
    else:
        return "N/A", "N/A"

def calculate_sector_times(split_times: List[str]) -> List[str]:
    sector_times = []
    previous_time = "0:00.000"

    for split_time in split_times:
        try:
            delta = datetime.strptime(split_time, "%M:%S.%f") - datetime.strptime(previous_time, "%M:%S.%f")
            sector_times.append(str(delta)[2:])  # Skip "0:" part in "0:XX.XXX" string
            previous_time = split_time
        except ValueError:
            sector_times.append("N/A")

    return sector_times

def extract_rider_info_all_pages(filename: str, table_start_line: int = 25) -> List[Dict[str, Union[str, List[str]]]]:
    doc = fitz.open(filename)
    riders_info = []
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")
        lines = text.split('\n')
        line_start = table_start_line

        while line_start < len(lines):
            rider_info = lines[line_start:line_start + 20]
            if len(rider_info) < 19:
                break

            # No team case
            if rider_info[5].isdigit():
                speed_trap, speed_trap_rank = extract_time_and_rank(rider_info[8])
                split_times, split_time_ranks = zip(*(extract_time_and_rank(s) for s in rider_info[9:13]))
                rider_data = {
                    'rank': rider_info[0].split()[0].replace('.', ''),
                    'protected': rider_info[0].split()[1] if len(rider_info[0].split()) > 1 else '',
                    'rider_number': rider_info[1].split()[0],
                    'name': ' '.join(rider_info[1].split()[1:]),
                    'team': 'N/A',
                    'uci_id': rider_info[5],
                    'country': rider_info[6],
                    'birth_year': rider_info[7],
                    'speed_trap': speed_trap,
                    'speed_trap_rank': speed_trap_rank,
                    'split_times': list(split_times),
                    'split_time_ranks': list(split_time_ranks),
                    'final_time': rider_info[13],
                    'gap': rider_info[17] if len(rider_info) > 17 else 'N/A',
                    'points': rider_info[18] if len(rider_info) > 18 else 'N/A'
                }
                next_offset = 19
            # With team case
            else:
                speed_trap, speed_trap_rank = extract_time_and_rank(rider_info[9])
                split_times, split_time_ranks = zip(*(extract_time_and_rank(s) for s in rider_info[10:14]))
                rider_data = {
                    'rank': rider_info[0].split()[0].replace('.', ''),
                    'protected': rider_info[0].split()[1] if len(rider_info[0].split()) > 1 else '',
                    'rider_number': rider_info[1].split()[0],
                    'name': ' '.join(rider_info[1].split()[1:]),
                    'team': rider_info[5],
                    'uci_id': rider_info[6],
                    'country': rider_info[7],
                    'birth_year': rider_info[8],
                    'speed_trap': speed_trap,
                    'speed_trap_rank': speed_trap_rank,
                    'split_times': list(split_times),
                    'split_time_ranks': list(split_time_ranks),
                    'final_time': rider_info[14],
                    'gap': rider_info[18] if len(rider_info) > 18 else 'N/A',
                    'points': rider_info[19] if len(rider_info) > 19 else 'N/A'
                }
                next_offset = 20

            if rider_data['final_time'] in ['DNF', 'DNS']:
                break

            sector_times = calculate_sector_times(rider_data['split_times'])
            rider_data['sector_times'] = sector_times
            riders_info.append(rider_data)
            line_start += next_offset

    return riders_info

# Generate DataFrame
riders_info = extract_rider_info_all_pages(filename, table_start_line)

df = pd.DataFrame(riders_info)
for i in range(4):
    df[f'split_{i+1}'] = df['split_times'].apply(lambda x: x[i] if len(x) > i else 'N/A')
    df[f'split_{i+1}_rank'] = df['split_time_ranks'].apply(lambda x: x[i] if len(x) > i else 'N/A')
    df[f'sector_{i+1}'] = df['sector_times'].apply(lambda x: x[i] if len(x) > i else 'N/A')

# Rank the sector times correctly
for i in range(4):
    df[f'sector_{i+1}_rank'] = df[f'sector_{i+1}'].apply(
        lambda x: timedelta(minutes=int(x.split(":")[0]), seconds=float(x.split(":")[1]))
        if x not in ["N/A", "-"]
        else timedelta.max
    ).rank(method="min").astype(int)

# Handle the final sector (sector_5)
df[f'sector_5'] = df.apply(lambda row: 
    str(datetime.strptime(row["final_time"], "%M:%S.%f") - datetime.strptime(row["split_4"], "%M:%S.%f"))[2:]
    if row["final_time"] not in ["DNF", "DNS", "N/A", "-"] and row["split_4"] not in ["DNF", "DNS", "N/A", "-"]
    else "N/A", axis=1)

df[f'sector_5_rank'] = df[f'sector_5'].apply(
    lambda x: timedelta(minutes=int(x.split(":")[0]), seconds=float(x.split(":")[1]))
    if x not in ["N/A", "-"]
    else timedelta.max
).rank(method="min").astype(int)

df.drop(columns=['split_times', 'split_time_ranks', 'sector_times'], inplace=True)

# Display and Save
display(df)
file_prefix = filename.split('/')[-1].split('.')[0]
df.to_csv(f'data/{file_prefix}.csv", index=False')


OverflowError: days=-1000000000; must have magnitude <= 999999999

Exception ignored in: 'pandas._libs.algos.rank_sorted_1d'
Traceback (most recent call last):
  File "algos.pyx", line 77, in pandas._libs.algos.are_diff
OverflowError: days=-1000000000; must have magnitude <= 999999999


OverflowError: days=-1000000000; must have magnitude <= 999999999

Exception ignored in: 'pandas._libs.algos.rank_sorted_1d'
Traceback (most recent call last):
  File "algos.pyx", line 77, in pandas._libs.algos.are_diff
OverflowError: days=-1000000000; must have magnitude <= 999999999


,rank,protected,rider_number,name,team,uci_id,country,birth_year,speed_trap,speed_trap_rank,...,sector_3,split_4,split_4_rank,sector_4,sector_1_rank,sector_2_rank,sector_3_rank,sector_4_rank,sector_5,sector_5_rank
0,1,P,1,BRUNI Loic,SPECIALIZED GRAVITY,10007544358,FRA,1994,44.944,18,...,00:35.209000,3:44.241,1,00:30.886000,1,1,3,5,00:24.090000,15
1,2,P,4,ILES Finn,SPECIALIZED GRAVITY,10090907774,CAN,1999,45.056,16,...,00:36.309000,3:47.685,2,00:31.028000,3,4,17,11,00:23.939000,5
2,3,P,10,SHAW Luca,CANYON CLLCTV FACTORY TEAM,10008813442,USA,1996,44.527,27,...,00:36.071000,3:48.081,5,00:30.865000,2,6,9,3,00:24.008000,7
3,4,P,8,BROSNAN Troy,CANYON CLLCTV FACTORY TEAM,10007307417,AUS,1993,46.124,2,...,00:35.819000,3:48.003,3,00:30.566000,4,9,7,1,00:24.237000,25
4,5,p,17,O CALLAGHAN Oisin *,YT MOB,10017486353,IRL,2003,45.802,4,...,00:35.600000,3:48.155,6,00:31.617000,22,3,4,31,00:24.162000,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,126,,140,LANCETT-EDWARDS Josh *,N/A,10124404706,GBR,2005,26.354,134,...,01:46.063000,7:25.822,127,01:39.881000,125,126,130,0,00:57.205000,127
126,127,,106,WILLIAMS Preston *,THE ALLIANCE,10088217743,GBR,2003,41.618,110,...,00:38.154000,9:04.396,130,00:33.797000,57,129,71,103,00:26.584000,110
127,128,,95,LAMM Nico,N/A,10074168507,GER,1999,28.016,132,...,01:39.668000,8:16.303,129,01:12.549000,98,128,129,126,01:31.847000,128
128,129,,143,MURRAY Charles,N/A,10084688054,NZL,1996,38.055,121,...,00:46.811000,12:57.837,131,00:46.851000,91,130,123,120,00:43.045000,123
